In [1]:
import numpy as np
import pandas as pd
import os
import datetime
from pandas import DataFrame as df
import time

In [2]:
src_path = 'data'

In [5]:
interface = ['s5','s6a','s6a_auc','s11']

In [19]:
files3 = os.listdir(src_path + '/' + 's6a_auc_call')

In [38]:
files_1 = os.listdir(src_path + '/s5_call')

In [39]:
s5 = pd.read_csv(src_path + '/s5_call' + '/' + files_1[0], encoding='cp949')

In [40]:
s5.columns

Index(['520030503598199', '66929862265', '2020-07-28 18:00:01.446491',
       '2020-07-28 18:00:01.840257', 'Create Session Request', '1', '92',
       '0.394', '35380010(미등록TAC)', 'Unnamed: 9', '118.235.204.81',
       '110.70.246.1', '110.49.180.53', '172.21.222.71', 'internet',
       'Unnamed: 15', '0', 'Unnamed: 17', '1.1', '0.1', 'Unnamed: 20', '0.2',
       '301', '131', '1.2', '1.3', '432'],
      dtype='object')

In [88]:
#def data_processing(): 데이터 전체 인터페이스 완료
files_all = [] ## 파일 리스트
min_list = []
start = time.time()
src_path = 'data/20200729' #######베이스 경로
s6a_data = os.listdir(src_path + '/s6a_call/20200729') ##s6a 경로
s6a_auc_data = os.listdir(src_path + '/s6a_auc_call') ## s6a auc 경로
s5_data = os.listdir(src_path + '/s5_call') ## s5
s11_data = os.listdir(src_path + '/s11_call') ## s11

######################################################기초 선언 리스트에 넣기

files_all.append(s6a_data) ## s6 
files_all.append(s6a_auc_data) #s6a auc
files_all.append(s5_data) ## s5
files_all.append(s11_data) ## s11

for n,files in enumerate(files_all):
    for i,file in enumerate(files):
        if int(n / 2) == 0: 
            if n == 0:
                interface = pd.read_csv(src_path + '/' + 's6a_call' + '/20200729/' + files[i], encoding='cp949', names = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25'])
            else:
                interface = pd.read_csv(src_path + '/' + 's6a_auc_call' + '/' + files[i], encoding='cp949', names = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25'])
            interface = interface[['3','4','13','18']]
            interface = interface.loc[interface['13'] == 1]
            interface = interface.drop(['13'], axis=1)                     

            print("#######################################")
            print("############fail 날리기 s6a, s6a auc################")
            print("#######################################")
            ########################################## success 날리기

            interface['4'] = pd.to_datetime(interface['4'])
            interface['count'] = 1
            interface = pd.pivot_table(interface, values='count',index=['4','18'],columns=['3'])
            interface.fillna(0, inplace=True)
            interface.columns.name = ''
            interface = interface.reset_index()

        else: 
            if n == 2:
                interface = pd.read_csv(src_path + '/' + 's5_call' + '/' + files[i], encoding='cp949', names = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26'])
            else:  
                interface = pd.read_csv(src_path + '/' + 's11_call' + '/' + files[i], encoding='cp949', names = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26'])           

            interface = interface[['3','4','5','18']]
            interface = interface.loc[interface['5']==1]
            interface = interface.drop(['5'], axis=1)

            print("#######################################")
            print("############fail 날리기 s5, s11################")
            print("#######################################")
            ########################################## success 날리기

            interface['3'] = pd.to_datetime(interface['3'])
            interface['count'] = 1
            interface = pd.pivot_table(interface, values='count',index=['3','18'],columns=['4'])
            interface.fillna(0, inplace=True)
            interface.columns.name = ''
            interface = interface.reset_index()

        ########################################Pivot

        start_time = pd.to_datetime(file.split('_')[0])

        temp0 = interface[interface['18'] == 0]
        temp1 = interface[interface['18'] == 1]

        min_data0 = pd.Series(temp0.sum(axis=0))
        min_data0['RESPONSE_TIME'] = start_time
        min_data0 = min_data0.reindex([list(min_data0.index)[-1]] + list(min_data0.index)[:-1])
        min_data0.rename(index={'18':'ROAMING'}, inplace=True)

        min_data1 = pd.Series(temp1.sum(axis=0))
        min_data1['RESPONSE_TIME'] = start_time
        min_data1 = min_data1.reindex([list(min_data1.index)[-1]] + list(min_data1.index)[:-1])
        min_data1.rename(index={'18':'ROAMING'}, inplace=True)
        min_data1['ROAMING'] = 1

        min_data_sum = pd.Series(interface.sum(axis=0))
        min_data_sum['RESPONSE_TIME'] = start_time
        min_data_sum = min_data_sum.reindex([list(min_data_sum.index)[-1]] + list(min_data_sum.index)[:-1])
        min_data_sum.rename(index={'18':'ROAMING'}, inplace=True)
        min_data_sum['ROAMING'] = "SUM"

        #######################################romming 나누기
        min_list.append(min_data0)
        min_list.append(min_data1)
        min_list.append(min_data_sum)

        if i == 0:
            pcd_data = pd.DataFrame(columns=interface.columns)

            if int(n / 2) == 0:
                pcd_data.rename(columns = {'4' : 'RESPONSE_TIME', '18' : 'ROAMING'}, inplace = True)

            else:
                pcd_data.rename(columns = {'3' : 'RESPONSE_TIME', '18' : 'ROAMING'}, inplace = True)

            for j in min_list:
                pcd_data = pcd_data.append(j, ignore_index=True)

        else:
            for j in min_list:
                pcd_data = pcd_data.append(j, ignore_index=True)

        min_list.clear()

        print("#######################################")
        print("처리완료 : ", i)
        print("#######################################")

    pcd_data.fillna(0, inplace=True)
    ###################################################인터페이스에 맞게 저장
    
    if int(n / 2) == 0:
        if n == 0:
            pcd_data.to_csv(files[0][:8] +'_s6a.csv', header=True, index=False)
            print("s6a 완료")
        else:
            pcd_data.to_csv(files[0][:8] +'_s6a_auc.csv', header=True, index=False)
            print("s6a_auc 완료")
    else:
        if n == 2:
            pcd_data.to_csv(files[0][:8] +'_s5.csv', header=True, index=False)
            print("s5 완료")
        else:
            pcd_data.to_csv(files[0][:8] +'_s11.csv', header=True, index=False)
            print("s11 완료")

    print("")
    print("@@@@@@@@@@@@@@@@@@@@@@@@하나 완료@@@@@@@@@@@@@@@@@@@@@@2")
    print("")
print("time :", int(time.time() - start))  

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  0
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  1
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  2
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  3
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
##############################

#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  38
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  39
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  40
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  41
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
##########################

#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  76
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  77
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  78
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  79
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
##########################

#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  114
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  115
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  116
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
#######################################
#######################################
처리완료 :  117
#######################################
#######################################
############fail 날리기 s6a, s6a auc################
######################

...

#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1249
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1250
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1251
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1252
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
###

#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1287
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1288
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1289
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1290
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
###

#######################################
처리완료 :  1325
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1326
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1327
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1328
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1329
#######################################
###################################

#######################################
처리완료 :  1363
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1364
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1365
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1366
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1367
#######################################
###################################

#######################################
처리완료 :  1401
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1402
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1403
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1404
#######################################
#######################################
############fail 날리기 s5, s11################
#######################################
#######################################
처리완료 :  1405
#######################################
###################################

In [102]:
t123 = pd.read_csv('20200728_s5.csv', encoding='cp949')

In [108]:
t123[4300:4320]

,RESPONSE_TIME,ROAMING,Create Bearer Request,Create Session Request,Delete Bearer Request,Delete Session Request,Modify Bearer Command,Update Bearer Request,Modify Bearer Request,Bearer Resource Command,Delete Bearer Command
4300,2020-07-29 23:52:00,1,0.0,3472.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0
4301,2020-07-29 23:52:00,SUM,4.0,3971.0,9.0,70.0,1.0,1.0,0.0,0.0,0.0
4302,2020-07-29 23:53:00,0.0,5.0,558.0,1.0,27.0,2.0,2.0,1.0,0.0,0.0
4303,2020-07-29 23:53:00,1,0.0,4233.0,0.0,24.0,0.0,2.0,0.0,0.0,0.0
4304,2020-07-29 23:53:00,SUM,5.0,4791.0,1.0,51.0,2.0,4.0,1.0,0.0,0.0
4305,2020-07-29 23:54:00,0.0,1.0,531.0,8.0,30.0,0.0,4.0,0.0,0.0,0.0
4306,2020-07-29 23:54:00,1,0.0,3487.0,0.0,31.0,0.0,1.0,0.0,0.0,0.0
4307,2020-07-29 23:54:00,SUM,1.0,4018.0,8.0,61.0,0.0,5.0,0.0,0.0,0.0
4308,2020-07-29 23:55:00,0.0,7.0,514.0,10.0,32.0,0.0,0.0,1.0,0.0,0.0
4309,2020-07-29 23:55:00,1,0.0,4584.0,0.0,27.0,1.0,0.0,0.0,0.0,0.0


In [89]:
os.chdir('C:\\Users\\user\\datapro\\데이터 전처리 완료\\20200729')

In [57]:
data_processing_s6a()

#######################################
############fail 날리기################
#######################################
#######################################
처리완료 :  0
#######################################
0


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


#######################################
############fail 날리기################
#######################################
#######################################
처리완료 :  1
#######################################
0
#######################################
############fail 날리기################
#######################################
#######################################
처리완료 :  2
#######################################
0
#######################################
############fail 날리기################
#######################################
#######################################
처리완료 :  3
#######################################
0


KeyboardInterrupt: 

In [49]:
df10 = pd.read_csv('20200728_s6a.csv', encoding='cp949')
df11 = pd.read_csv('20200728_s6a_auc.csv', encoding='cp949')

In [55]:
df11

,RESPONSE_TIME,ROAMING,Authentication-Information_3002,Authentication-Information_4001,Authentication-Information_4181,Authentication-Information_5001,Authentication-Information_5003,Authentication-Information_5420,Authentication-Information_5421,Authentication-Information_Timeout,Authentication-Information_5012,Authentication-Information_3003,Authentication-Information_3004,Authentication-Information_0
0,2020-07-28 18:00:00,0.0,0.0,2.0,0.0,1297.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0
1,2020-07-28 18:00:00,1.0,1.0,0.0,2.0,17.0,38.0,59.0,6.0,6.0,0.0,0.0,0.0,0.0
2,2020-07-28 18:00:00,SUM,1.0,2.0,2.0,1314.0,38.0,59.0,6.0,18.0,0.0,0.0,0.0,0.0
3,2020-07-28 18:01:00,0.0,0.0,1.0,0.0,1293.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0
4,2020-07-28 18:01:00,1,1.0,0.0,0.0,23.0,17.0,84.0,4.0,3.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,2020-07-28 23:46:00,1,0.0,0.0,0.0,17.0,14.0,65.0,1.0,0.0,1.0,0.0,0.0,0.0
1040,2020-07-28 23:46:00,SUM,0.0,4.0,7.0,900.0,14.0,65.0,1.0,4.0,1.0,0.0,0.0,0.0
1041,2020-07-28 23:47:00,0.0,0.0,3.0,0.0,861.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
1042,2020-07-28 23:47:00,1,0.0,0.0,0.0,18.0,32.0,60.0,6.0,0.0,8.0,0.0,0.0,0.0


In [61]:
print(os.path.abspath(os.curdir))

C:\Users\user


In [62]:
os.chdir("C:\Users\user\데이터전처리")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-62-58b64669ecaa>, line 1)

In [35]:
print(int(n / 2))

1


In [34]:
n = 3

In [26]:
if n / 2 == 0:
    print("출력")

In [26]:
pd.Series(temp1.sum(axis=0))


18                             387.0
Cancel-Location_Timeout          0.0
Insert-Subscriber-Data_5001      0.0
Notify_0                         0.0
Notify_5001                      2.0
Notify_5012                      6.0
Notify_5423                      0.0
Notify_Timeout                   1.0
Purge-UE_5001                    3.0
Purge-UE_Timeout                 0.0
Update-Location_5004            57.0
Update-Location_5012           260.0
Update-Location_5420            50.0
Update-Location_5421             3.0
Update-Location_Timeout          8.0
dtype: float64

In [29]:
temp0['Cancel-Location_Timeout'].sum()

5.0

In [30]:
temp1['Cancel-Location_Timeout'].sum()

0.0

In [209]:
#s5 전처리
#################################################################################################################################    
mcalltype_list = ['Bearer Resource Command_69.0','Create Bearer Request_110.0','Create Bearer Request_69.0','Create Bearer Request_72.0',
              'Create Bearer Request_73.0','Create Bearer Request_87.0','Create Bearer Request_88.0','Create Bearer Request_89.0',
              'Create Bearer Request_94.0','Create Session Request_205.0','Create Session Request_209.0','Create Session Request_68.0',
              'Create Session Request_72.0','Create Session Request_73.0','Create Session Request_78.0','Create Session Request_83.0',
              'Create Session Request_84.0','Create Session Request_92.0','Create Session Request_93.0','Create Session Request_94.0',
              'Delete Bearer Command_64.0','Delete Bearer Request_100.0','Delete Bearer Request_64.0','Delete Bearer Request_72.0',
              'Delete Session Request_109.0','Delete Session Request_64.0','Modify Bearer Command_64.0','Modify Bearer Command_68.0',
              'Modify Bearer Command_89.0','Modify Bearer Request_64.0','Update Bearer Request_100.0','Update Bearer Request_64.0',
              'Update Bearer Request_72.0','Update Bearer Request_73.0','Update Bearer Request_76.0','Update Bearer Request_77.0',
              'Update Bearer Request_87.0','Update Bearer Request_89.0','Update Bearer Request_94.0','etc']
mcalltype_list = sorted(list(set(mcalltype_list)))

mmeips = ['118.235.200.145','118.235.200.17','118.235.201.209','118.235.201.81',
      '118.235.202.145','118.235.202.17','118.235.202.81','118.235.203.145',
      '118.235.203.17','118.235.203.81','118.235.204.145','118.235.204.17',
      '118.235.204.209','118.235.204.81','118.235.205.145','118.235.205.17',
      '118.235.205.81','118.235.221.65','61.252.41.145','61.252.41.81']

NEW_FC = []
rst_datas = {}
sep_mme = {}
pcd_data = {}

###############################################################################################################################
src_path = 'data'
files = os.listdir(src_path + '/s5_call')
count = 0

for i,file in enumerate(files):

    print('---------------------------------------------------')
    print(file, '처리중...')
    print('---------------------------------------------------')


    #file = file.split('_')[0]
    s5 = pd.read_csv(src_path + '/s5_call' + '/' + file, encoding='cp949', names = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26'])
    s5 = s5[['3','4','5','6','10','18']]
    s5 = s5.loc[s5['5']==1]
    s5.drop('5', axis=1, inplace=True)
    #s6a.to_csv(file +'_s6a.csv', header=False, index=False)

    s5["MTYPE_FC"] = s5["4"] + '_' + s5["6"].map(str)
    
    s5['3'] = pd.to_datetime(s5['3'])  #시간 형식 으로 바꿈
    s5.drop(['4', '6'], axis=1, inplace=True)   
    ######################################################################################################

    if len(NEW_FC) == 0:
        NEW_FC = list(set(s5.loc[~s5['MTYPE_FC'].isin(mcalltype_list), 'MTYPE_FC']))
    else:
        NEW_FC = list(set(NEW_FC + list(set(NEW_FC) - set(list(s5.loc[~s5['MTYPE_FC'].isin(mcalltype_list), 'MTYPE_FC'])))))

    #####################################################################################################       

    s5.loc[~s5['MTYPE_FC'].isin(mcalltype_list), 'MTYPE_FC'] = 'etc'

    for mmeip in mmeips:
        
        start_time = pd.to_datetime(file.split('_')[0]) + datetime.timedelta(seconds=4) ###스타트 타임 설정
        
        tmp_df = s5[s5['10'] == mmeip]

        if len(tmp_df) == 0:
            sep_mme[mmeip] = s5.loc[[0]]
            sep_mme[mmeip].drop('10', axis=1, inplace=True)
            sep_mme[mmeip]['COUNT'] = 0
            print(mmeip, "여기돌았다.")
        else:
            sep_mme[mmeip] = tmp_df
            sep_mme[mmeip].drop('10', axis=1, inplace=True)
            sep_mme[mmeip]['COUNT'] = 1
    
        sep_mme[mmeip] = pd.pivot_table(sep_mme[mmeip], values='COUNT', index=['3','18'], columns=['MTYPE_FC'])
    
        sep_mme[mmeip] = sep_mme[mmeip].reindex(columns=mcalltype_list)
        sep_mme[mmeip].columns.name = ''
        sep_mme[mmeip].fillna(0, inplace=True)
        sep_mme[mmeip] = sep_mme[mmeip].reset_index()
        sep_mme[mmeip].rename(columns={'3':'RESPONSE_TIME', '18':'ROMMING'}, inplace = True)

        #print(file.split('_')[0])  ###확인
        
        #end_time = start_time + datetime.timedelta(minutes=1)
        #sep_mme[mmeip] = sep_mme[mmeip][sep_mme[mmeip]['3'].between(start_time, end_time)]
        pcd_data[mmeip] = pd.DataFrame(columns=sep_mme[mmeip].columns)

        #print(temp_time)
        temp = sep_mme[mmeip]
        temp0 = temp[temp['ROMMING'] == 0]
        temp1 = temp[temp['ROMMING'] == 1]

        min_data0 = pd.Series(temp0.sum(axis=0))
        min_data0['RESPONSE_TIME'] = start_time
        min_data0 = min_data0.reindex([list(min_data0.index)[-1]] + list(min_data0.index)[:-1])
        
        
        min_data1 = pd.Series(temp1.sum(axis=0))
        min_data1['RESPONSE_TIME'] = start_time
        min_data1 = min_data1.reindex([list(min_data1.index)[-1]] + list(min_data1.index)[:-1])
        min_data1['ROMMING'] = 1

        min_data_sum = pd.Series(temp.sum(axis=0))
        min_data_sum['RESPONSE_TIME'] = start_time
        min_data_sum = min_data_sum.reindex([list(min_data_sum.index)[-1]] + list(min_data_sum.index)[:-1])
        min_data_sum['ROMMING'] = 'SUM'

        pcd_data[mmeip] = pcd_data[mmeip].append(min_data0, ignore_index=True)        
        pcd_data[mmeip] = pcd_data[mmeip].append(min_data1, ignore_index=True)
        pcd_data[mmeip] = pcd_data[mmeip].append(min_data_sum, ignore_index=True)
        
        if len(rst_datas) < 20:
            rst_datas[mmeip] = pcd_data[mmeip]
        else:
            rst_datas[mmeip] = pd.concat([rst_datas[mmeip], pcd_data[mmeip]])
        count += i

print("*****************************************")
print(count,"완료")
print("*****************************************")

---------------------------------------------------
202007281800_s5_call_fail.dat 처리중...
---------------------------------------------------


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


61.252.41.81 여기돌았다.
---------------------------------------------------
202007281801_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281802_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281803_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281804_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281805_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281806_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
--------------

61.252.41.81 여기돌았다.
---------------------------------------------------
202007281852_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281853_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281854_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281855_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281856_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281857_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
--------------

61.252.41.81 여기돌았다.
---------------------------------------------------
202007281943_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281944_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281945_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281946_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281947_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007281948_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
--------------

61.252.41.81 여기돌았다.
---------------------------------------------------
202007282034_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282035_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282036_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282037_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282038_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282039_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
--------------

61.252.41.81 여기돌았다.
---------------------------------------------------
202007282125_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282126_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282127_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282128_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282129_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282130_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
--------------

61.252.41.81 여기돌았다.
---------------------------------------------------
202007282216_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282217_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282218_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282219_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282220_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282221_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
--------------

61.252.41.81 여기돌았다.
---------------------------------------------------
202007282307_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282308_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282309_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282310_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282311_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
---------------------------------------------------
202007282312_s5_call_fail.dat 처리중...
---------------------------------------------------
61.252.41.81 여기돌았다.
--------------

In [ ]:
test1 = pd.Series()